In [4]:
from pydantic import BaseModel, Field
from typing import List

class Slide(BaseModel):
    title: str = Field(
        description="title for this slide.",
    )
    text: str = Field(
        description="text for this slide.",
    )

class Slides(BaseModel):
    slides: List[Slide] = Field(
        description="Slides to present the document in a presentation.",
    )

In [13]:
with open("texts/sample.txt", "r") as f:
    doc =f.read()

In [14]:
from google import genai
from pydantic import BaseModel, Field
from typing import List, Optional

client = genai.Client()

prompt = """
make slides for the following text.
{text}
"""

response = client.models.generate_content(
    model="gemini-3-flash-preview",
    contents=prompt.format(text=doc),
    config={
        "response_mime_type": "application/json",
        "response_json_schema": Slides.model_json_schema(),
    },
)

res = Slides.model_validate_json(response.text)
print(res)

slides=[Slide(title='OpenAI: From Idealistic Non-Profit to Global AI Leader', text="An overview of OpenAI's transformation since its 2015 launch, covering its original mission, key milestones, and the evolving debate over AI safety and openness."), Slide(title='The Founding Vision (December 2015)', text='Launched as a non-profit research lab unconstrained by financial returns. Mission: Advance digital intelligence to benefit humanity. Key features: $1B in funding, top-tier team (Sutskever, Karpathy, etc.), and a commitment to openness and safety.'), Slide(title='Early Community Reactions', text="Initial themes included excitement over the star-studded team and debates on whether deep learning would plateau. Discussions touched on AI safety vs. 'fear-mongering,' the ethics of AI, and future societal impacts like job loss and propaganda."), Slide(title='2016–2018: The Open Research Phase', text='Focused on transparent research and community tools. Key releases included OpenAI Gym, Univer

In [16]:
len(res.slides)

11

In [18]:
import os
import requests
from dotenv import load_dotenv
load_dotenv()

OPENROUTER_API_KEY = os.environ["OPENROUTER_API_KEY"]


In [ ]:
def generate_structured_output(prompt: str, model: str, schema: BaseModel) -> dict:
    url = "https://openrouter.ai/api/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": model,
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "response_format": {"type": "json_schema", "json_schema": schema.model_json_schema()}
    }
    response = requests.post(url, headers=headers, json=payload)
    result = response.json()
    return result

In [25]:
Slides.model_json_schema()

{'$defs': {'Slide': {'properties': {'title': {'description': 'title for this slide.',
     'title': 'Title',
     'type': 'string'},
    'text': {'description': 'text for this slide.',
     'title': 'Text',
     'type': 'string'}},
   'required': ['title', 'text'],
   'title': 'Slide',
   'type': 'object'}},
 'properties': {'slides': {'description': 'Slides to present the document in a presentation.',
   'items': {'$ref': '#/$defs/Slide'},
   'title': 'Slides',
   'type': 'array'}},
 'required': ['slides'],
 'title': 'Slides',
 'type': 'object'}

In [30]:
res = generate_structured_output(prompt.format(text=doc), "google/gemini-3-flash-preview", Slides)

In [31]:
res

{'id': 'gen-1766131009-trIqXoLxZgqpj22gMYRS',
 'provider': 'Google AI Studio',
 'model': 'google/gemini-3-flash-preview',
 'object': 'chat.completion',
 'created': 1766131009,
 'choices': [{'logprobs': None,
   'finish_reason': 'stop',
   'native_finish_reason': 'STOP',
   'index': 0,
   'message': {'role': 'assistant',
    'content': '## OpenAI: From Idealistic Non-Profit to Global Powerhouse (2015–2024)\n\n### Slide 1: Dec 2015: The Launch of OpenAI\n*   **The Mission:** A non-profit research lab "unconstrained by financial return."\n*   **The Goal:** Advance digital intelligence to benefit humanity.\n*   **Key Pillars:**\n    *   **Openness:** Sharing papers, code, and patents.\n    *   **Safety:** Ensuring AI is an extension of human will, broadly distributed.\n*   **The Star Team:** Founding talent including Ilya Sutskever, Andrej Karpathy, and Greg Brockman.\n*   **The Backing:** $1B in commitments from Elon Musk, Sam Altman, Peter Thiel, AWS, and Reid Hoffman.\n\n---\n\n### Slid

In [24]:
Slides.model_validate_json(res["choices"][0]["message"]["content"])

ValidationError: 1 validation error for Slides
  Invalid JSON: expected value at line 1 column 1 [type=json_invalid, input_value='## **Presentation: OpenA..., if not their methods.', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/json_invalid